In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import ssl
import os
import re

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [ ]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [ ]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [ ]:


def get_html_from_url(url):
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver') 
  browser.set_page_load_timeout(30) 
  browser.get(url)
   
  last_height = browser.execute_script('return document.body.scrollHeight')

  flag=1

  while flag==1:
    
    try:
       browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infine_scroll(last_height))
       last_height = new_height

    except:
        print("End of page reached")
        flag = 0
  html = browser.page_source
  return html

In [ ]:
#helper function get first number in string

def str_first_num(s):
    first_num = ''
    flag_digit=False
    for el in s: 
        if el.isdigit():
            first_num += el
            flag_digit = True
        else:
            if flag_digit:
                break
    #If there are no upvotes replace '' with 0
    if first_num == '':
        first_num = 0
    return first_num
    

In [ ]:
def unpack_list(li):
    unpacked_list = []
    for el in li:
        if isinstance(el, list):
            unpacked_list.extend([*el])
        elif isinstance(el, str):
            unpacked_list.extend([el])
    return unpacked_list


In [51]:
# a helper function that removes unwanted words/sections like url links
def strip_rgx_words(inp, regex_ = ['image','url']):
    #remove urls or images
    rgx_word = None
    for rgx in regex_:
      if rgx in inp:
          rgx_word = rgx
          break
    if rgx_word is not None:
      out = inp.split(rgx_word)[0]
    else:
      out =inp
    return out

In [52]:
#read question urls from csv file
src_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\quet_indexes.csv"
df = pd.read_csv(src_path)

In [53]:
#get list of urls
url_list = list(df['url'].values)

In [54]:
url_list

['https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/How-long-could-the-brain-survive-theoretically-if-we-had-the-technology-to-replace-all-other-body-parts-as-you-aged-with-functioning-organs-that-are-grown-in-a-lab-using-your-DNA',
 'https://www.quora.com/What-are-the-top-10-emerging-technologies-in-the-next-5-10-years-2020%E2%80%932025',
 'https://www.quora.com/What-are-the-upcoming-emerging-technologies-in-software-industry',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-most-advanced-technologies-that-people-dont-know-about-yet',
 'https://www.quora.com/When-will-Fusion-reactors-become-a-reality',
 'https://www.quora.com/In-future-IOT-Internet-of-things-is-trending-technology-or-not',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-solutions-to-emerging-issues-in-communication']

In [108]:
#get followers count based on names
def get_person_followers_count(names):
    followers_count = []
    for name in names:
        name = name.replace(' ', '-')
        url_name = 'https://www.quora.com/profile/' + name
        browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver') 
        browser.set_page_load_timeout(30) 
        browser.get(url_name)
        name_html = browser.page_source
        name_soup = BeautifulSoup(name_html)
        name_followers = name_soup.find('div', {'class':"q-text qu-dynamicFontSize--small qu-color--gray"})
        try:
            name_followers = name_followers.find_next(text=True).strip()
        except:
            name_followers = 'UNKNOWN'
        followers_count.append(name_followers)
    return followers_count
    
def fill_list_counts(inf):
    answers_upvotes = []
    for ans_inf in inf:
        if ans_inf == []:
           val = 0
        else:
            val = ans_inf[0].find_next(text=True).strip()
        answers_upvotes.append(val)
    return answers_upvotes

def fill_list_str(inf):
    answers_upvotes = []
    for ans_inf in inf:
        if ans_inf[0] == []:
           val = 'UNKNOWN'
        else:
            val = ans_inf[0].find_next(text=True).strip()
        answers_upvotes.append(val)
    return answers_upvotes


def answers_into_df(url):
    '''input: url for scrapping answers to a particular question.
        out: df with answers for each url/question'''
    #request url html
    
    #create soup object with htmal parser
    page_html = get_html_from_url(url)
    soup = BeautifulSoup(page_html)

    boxes = soup.find_all('div', {'class':"q-box qu-pt--medium qu-hover--bg--darken"})
  
    answers = [box.find_all('p', {'class':"q-text qu-display--block qu-wordBreak--break-word qu-textAlign--start"}) for box in boxes]
    answers_text = []
    for ans in answers:
        #if there are more than one element in list unpack all into one list         
        if len(ans) > 1:
            ans_text_list = [a.find_next(text=True).strip() for a in ans] 
            ans_text_whole = ''.join(ans_text_list)
        elif isinstance(ans, list):
            if ans == []:
                ans_text_whole = 'UNKNOWN'
                continue
            try:
                ans_text_whole = ans[0].find_next(text=True).strip()
            except:
                ans_text_whole = ans[0]
        else:
            ans_text_whole = ans
        answers_text.append(ans_text_whole)


    #get a list of answer author upvotes and shares
    #answers_info = [box.find_all('button', {'role':'button'}) for box in boxes]
    answers_upvotes_info = [box.find_all('div', {'class':'q-text qu-overflow--hidden qu-display--inline-flex qu-ml--tiny qu-minHeight--20 qu-color--gray qu-minWidth--20'}) for box in boxes]
    answers_upvotes = []
    
    #answers upvotes:
    answers_upvotes = fill_list_counts(answers_upvotes_info)
    
    names_info = [box.find_all('div', {'class':'q-inlineFlex qu-alignItems--center qu-wordBreak--break-word'}) for box in boxes]
    answers_name = fill_list_str(names_info)
    
    names_follower_counts = get_person_followers_count(answers_name)
      
    author_counts = [box.find_all('span', {'class':"q-text qu-bold"}) for box in boxes]

    author_answers_count = []
    answer_views = []
        
    for i, aut_co in enumerate(author_counts):
        try:
            ans_co = aut_co[0].find_next(text=True).strip()
            ans_view = aut_co[1].find_next(text=True).strip()
        except:
            ans_co = 0
            ans_view = 0
        author_answers_count.append(ans_co)
        answer_views.append(ans_view)

    #create df with nans to fill it later with values
    columns = ["Answer",  "name", "followerCount", "upvoteCount", "answerCount", "answerViews"]

    x_shape = (len(answers), len(columns))
    x = np.tile(np.nan, x_shape)
   
    answers_df = pd.DataFrame(x, columns = columns)

    answers_df['Answer'] = pd.DataFrame(answers_text)   
    answers_df['name'] = pd.DataFrame(answers_name)
    answers_df['followerCount'] = pd.DataFrame(names_follower_counts)
    answers_df['upvoteCount'] = pd.DataFrame(answers_upvotes)
    answers_df['answerCount'] = pd.DataFrame(author_answers_count)
    answers_df['answerViews'] = pd.DataFrame(answer_views)

    return answers_df 

In [107]:
#show one example
answers_into_df(url_list[-1])

C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached


C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\1154510379.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


,Answer,name,upvoteCount,answerCount,answerViews,followerCount
0,Customer Communication Trends,Hope Seeker,9,0,0,0 followers
1,I can tell you what worked for me.I was a non-...,Curious,2.7K,104,598.8K,UNKNOWN
2,"From time to time, we hear from curious users ...",Grammarly,999,0,0,"2,405 followers"
3,There are 3 approaches.Option 1 : Myanmar to a...,Warren Wai,99,654,1.7M,0 followers
4,What do you think is the main communication pr...,Kirk Scott,1K,12.7K,63.2M,1 follower
5,The best solution currentlyThe best solution c...,Frank Dauenhauer,99,38K,116.2M,"36,887 followers"
6,Four things:Four things:India will never be di...,Rahul Chauhan,4K,2.5K,100.1M,34 followers
7,Here are some ways I use technology in my clas...,Naman Chakraborty,6.9K,342,36.2M,0 followers
8,The best option for finding a long-term stream...,Matthew Bates,99,9.7K,284.8M,4 followers
9,For one reason only: the crowds. Developed cou...,Carolina Silva,2.2K,0,0,39 followers


In [109]:
#columns = ["Answer", "upvoteCount", "answerCount", "followerCount", "name"]
columns = ["Answer", "followerCount", "name", "upvoteCount", "answerCount", "answerViews"]
df = pd.DataFrame(data = [], columns = columns)
questions = []

for i, url in enumerate(url_list):
    temp_df = answers_into_df(url)
    questions.append([url]*len(temp_df))
    df = df.append(temp_df)




C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached


C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\3057289583.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')
C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\2629752043.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached


C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\3057289583.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')
C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\2629752043.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


...

C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\3057289583.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')
C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\2629752043.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached


C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\3057289583.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')
C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\2629752043.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached


C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\3057289583.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')
C:\Users\Ravit\AppData\Local\Temp\ipykernel_18572\2629752043.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)


In [126]:
questions = unpack_list(questions)


In [128]:
df['question'] = pd.DataFrame(questions)


In [129]:
df.reset_index(inplace=True)


In [130]:
df.set_index('index')
df.drop('index', axis=1, inplace=True)

In [131]:
df

,Answer,followerCount,name,upvoteCount,answerCount,answerViews,question
0,50 years is aLong distance travel will be by h...,"45,199 followers",David Seidman,999,3.6K,82.9M,https://www.quora.com/What-will-the-worlds-tec...
1,Faster elevators capable of moving sideways.Fa...,"4,975 followers",Bobby Tatro,3.9K,2.1K,44.7M,https://www.quora.com/What-will-the-worlds-tec...
2,the only answer I can give you is that in that...,2 followers,Phillip Jensen,9,0,0,https://www.quora.com/What-will-the-worlds-tec...
3,"Well, the first question will be:How will glob...",1 follower,Frank Robertson,9,585,1.2M,https://www.quora.com/What-will-the-worlds-tec...
4,Go back to 1971 and describe in detail present...,"2,454 followers",Edward Lafreniere,99,12.2K,2M,https://www.quora.com/What-will-the-worlds-tec...
...,...,...,...,...,...,...,...
634,2020 will leave its mark in history as the yea...,1 follower,Debabrata,9,0,0,https://www.quora.com/What-are-the-solutions-t...
635,The biggest problem I see with communication i...,UNKNOWN,MaxEd,9,0,0,https://www.quora.com/What-are-the-solutions-t...
636,"Over the years, technology was being infused i...",651 followers,Almarie Meyer,9,1K,955.5K,https://www.quora.com/What-are-the-solutions-t...
637,Universities that apply technology in all its ...,0 followers,Hritvik Gaur,9,219,315.9K,https://www.quora.com/What-are-the-solutions-t...


In [133]:
#save df to csv
save_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info"
final_path = os.path.join(save_path, "ET_RCS_28SEP15_quora_data_emerging_technologies.csv")
df.to_csv(final_path)

In [ ]:
#TODO -  scrape all pages from this urls,
        #scrape all groups related to technology trends
     

         